In [1]:
#Import Libraries
import subprocess
import re
import pandas as pd
import os
import collections
import scipy.stats
import numpy as np
import scipy
import csv
import math
import seaborn as sns

from functools import reduce
from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline
from pandas.tools.plotting import table
from scipy import interp
from sklearn import svm
from sklearn.svm import SVC
from sklearn.datasets import make_blobs, make_classification


from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, GridSearchCV, LeaveOneOut, train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, auc, roc_curve, precision_recall_curve, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.multiclass import unique_labels
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D

In [2]:
dset_res=pd.read_csv("clintab_GMQL/miRNA/res_miRNA/res_miRNA_new.csv", delimiter='\t')
dset_senl=pd.read_csv("clintab_GMQL/miRNA/sl_miRNA_32/sl_miRNA_new.csv", delimiter='\t')
dset_sens=pd.read_csv("clintab_GMQL/miRNA/ss_miRNA_32/ss_miRNA_new.csv", delimiter='\t')

In [3]:
resdf=dset_res.pivot(index="patient", columns="mirna_id", values="rpm")
genes = resdf.columns.tolist()
resdf.shape

(60, 1881)

In [4]:
senldf=dset_senl.pivot(index="patient", columns="mirna_id", values="rpm")
senldf = senldf[genes]
senldf.shape

(34, 1881)

In [5]:
sensdf=dset_sens.pivot(index="patient", columns="mirna_id", values="rpm")
sensdf = sensdf[genes]
#sensdf.shape

In [6]:
resdf_median = resdf.median(0)
senldf_median = senldf.median(0)
sensdf_median = sensdf.median(0)

In [7]:
resdf_median.shape

(1881,)

In [8]:
print(len(resdf_median), len(senldf_median), len(sensdf_median))

1881 1881 1881


In [9]:
pvalues = []
for gene in tqdm(genes):
    try:
        gene_pvalue = scipy.stats.mannwhitneyu(sensdf[gene],senldf[gene]).pvalue
    except Exception:
        gene_pvalue = 1
    pvalues.append(gene_pvalue)
       
pvalues = pd.DataFrame({
    "gene": genes,
    #"resistant_median": resdf_median.values,
    "sensitive_long_median": senldf_median.values,
    "sensitive_short_median": sensdf_median.values,
    "mannwhiteney_pvalue": pvalues
})

pvalues.head()

100%|██████████| 1881/1881 [00:01<00:00, 1648.01it/s]


,gene,sensitive_long_median,sensitive_short_median,mannwhiteney_pvalue
0,hsa-let-7a-1,24112.802719,26649.792598,0.081465
1,hsa-let-7a-2,24083.507960,26780.061935,0.081465
2,hsa-let-7a-3,24323.474442,26721.749378,0.079307
3,hsa-let-7b,73081.638557,79906.082243,0.069808
4,hsa-let-7c,13086.891246,16183.028840,0.176760


In [12]:
len(pvalues)

1881

## Bonferroni

In [24]:
threshold = 0.5

significant_genes = pvalues[(pvalues.mannwhiteney_pvalue < threshold)].copy()
significant_genes = significant_genes.sort_values("mannwhiteney_pvalue", ascending=True)
print(significant_genes.shape[0])

1432


In [379]:
id_name=[]
gene_name=[]

for x in ["-".join(x.strip().split("-")[:1])  for x in (significant_genes['gene'])]:
    id_name.append(x)   
for x in ["-".join(x.strip().split("-")[1:])  for x in (significant_genes['gene'])]:
    gene_name.append(x) 
    
significant_genes['mirna_id']=id_name
significant_genes['gene_symbol']=gene_name
significant_genes=pd.DataFrame(significant_genes)

In [380]:
significant_genes.head()

,gene,resistant_median,sensitive_long_median,mannwhiteney_pvalue,mirna_id,gene_symbol
149,hsa-mir-1301,67.507454,126.137998,0.000022,hsa,mir-1301
795,hsa-mir-4446,0.000000,0.082939,0.000821,hsa,mir-4446
466,hsa-mir-3200,36.230009,79.711688,0.000857,hsa,mir-3200
888,hsa-mir-4524a,0.000000,0.000000,0.000992,hsa,mir-4524a
780,hsa-mir-4435-1,0.000000,0.000000,0.001006,hsa,mir-4435-1


In [17]:
n_res=len(dset_res['patient'].unique())
n_sens=len(dset_sens['patient'].unique())
n_senl=len(dset_senl['patient'].unique())

In [260]:
p_value_corr =[]

#for index, value in tqdm(significant_genes['gene'].iteritems()): 
for index, value in tqdm(pvalues['gene'].iteritems()):
        
        somma=len(pvalues)
        #somma=n_res+n_sens
        p_value=pvalues[pvalues['gene']==value]['mannwhiteney_pvalue'].iloc[0]
        p_value_corr.append(somma*p_value)
        #print(p_value_corr)
print(len(p_value_corr))

1881it [00:01, 1028.89it/s]

1881


In [261]:
#significant_genes['p_value_corr']=p_value_corr
pvalues['p_value_corr']=p_value_corr

In [262]:
threshold = 0.05

significant_genes_corr = pvalues[(pvalues.p_value_corr < threshold)].copy()
significant_genes_corr = significant_genes_corr.sort_values("mannwhiteney_pvalue", ascending=True)
print(significant_genes_corr.shape[0])
#significant_genes_corr.to_csv('new_data/miRNA/sl_ss_bonf_miRNA.csv', sep=',', header=True, index=False)
#significant_genes_corr.to_csv('new_data/miRNA/r_ss_bonf_miRNA.csv', sep=',', header=True, index=False)
#significant_genes_corr.to_csv('new_data/miRNA/r_sl_bonf_miRNA.csv', sep=',', header=True, index=False)

0


In [252]:
significant_genes_corr.head()

,gene,resistant_median,sensitive_long_median,mannwhiteney_pvalue,p_value_corr
149,hsa-mir-1301,67.507454,126.137998,0.000022,0.040941


In [ ]:
listageni=significant_genes_corr['gene'].values
#listageni

## Plot

In [ ]:
for g in listageni:

    boxplot1=pd.DataFrame(senldf, columns=[g])
    boxplot2=pd.DataFrame(sensdf, columns=[g])
    fig, ax= plt.subplots()
    boxplot3=pd.concat([boxplot1, boxplot2], axis=1)
    ax.set_title('Senl vs Sens gene: '+ g, fontsize = 10)

    boxplot_tot=boxplot3.boxplot(fontsize = 10)
    
    #print(boxplot3)
    plt.savefig('new_data/miRNA/boxplot/sl_ss/boxplot{}.png'.format(g))

In [ ]:
sns.set(style="whitegrid", font_scale=1.5)

for g in listageni[:]:
    data = pd.DataFrame(columns=["Values", "Class"])
    for row in senldf[g]:
        data = data.append({"Values":row,"Class":"SenL"}, ignore_index=True)
    for row in sensdf[g]:
        data = data.append({"Values":row,"Class":"SenS"}, ignore_index=True)
    
    plt.figure()
    sns.violinplot(data=data, x="Class", y="Values", palette="muted", saturation=0.75, alpha=0.7)
    plt.title(g)
    plt.savefig('new_data/miRNA/violinplot/sl_ss/violinplot{}.png'.format(g))

## Standard deviation

In [ ]:
r_std=resdf.std()
sl_std=senldf.std()
ss_std=sensdf.std()

#print (r_std)

In [ ]:
x=r_std.sort_values(axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
y=ss_std.sort_values(axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
z=sl_std.sort_values(axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [ ]:
res_std=x.to_frame()
res_std['genes']=res_std.index
res_std.index=np.arange(len(res_std))
#res_std

In [ ]:
sens_std=y.to_frame()
sens_std['genes']=sens_std.index
sens_std.index=np.arange(len(sens_std))
#sens_std

In [ ]:
senl_std=z.to_frame()
senl_std['genes']=senl_std.index
senl_std.index=np.arange(len(senl_std))
#senl_std

In [ ]:
threshold = 100.00

significant_std =y[(y > threshold)].copy()
significant_std = significant_std.sort_values(ascending=True)
print(significant_std.shape[0])

In [ ]:
w=significant_std.to_frame()

In [ ]:
w['genes']=w.index
w.index=np.arange(len(w))
#w

In [ ]:
#controllare quali sono i geni che risultano salvi tra boxplot, std, correzioni da test multiplo (qua
#basterebbe anche solo Bonferroni ma se poi me ne salva troppo pochi è tutt'un niente), fare 
#l'intersezione per definire le features, creare un nuovo dset contenente:
#solo le features, e la loro posizione (chrom, start, stop da dare a Sara)

In [ ]:
lista1=set(w['genes'])
lista2=set(significant_genes_corr ['gene'])

In [ ]:
inters=lista1.intersection(lista2)

In [ ]:
inters

## Appartenenza geni-classe

In [ ]:
gene_sl_ss=pd.read_csv("new_data/miRNA/sl_ss_bonf_miRNA.csv", delimiter=',')
#gene_r_ss=pd.read_csv("new_data/miRNA/r_ss_bonf_miRNA.csv", delimiter=',')
gene_r_sl=pd.read_csv("new_data/miRNA/r_sl_bonf_miRNA.csv", delimiter=',')

In [ ]:
gene_sl_ss.head()

In [ ]:
lista3=set(gene_sl_ss['gene'])
#lista4=set(gene_r_ss['gene'])
lista5=set(gene_r_sl['gene'])

In [ ]:
inters_=lista3.intersection(lista5)

In [ ]:
inters_

## FDR

In [321]:
#p_sorted=pvalues.sort_values(by='mannwhiteney_pvalue')

In [14]:
p_sorted=significant_genes.sort_values(by='mannwhiteney_pvalue')

In [15]:
a=p_sorted['mannwhiteney_pvalue'].get_values()

In [16]:
enumerate(a)

In [18]:
#num_total_tests=len(a)
num_total_tests=n_senl+n_sens
num_total_tests

141

In [19]:
def calc_benjamini_hochberg_corrections(p_values, num_total_tests):
   
    p_value_bh = []
    for i, p_value in enumerate(p_values):
        bh_value = p_value * num_total_tests / (i + 1)
        #bh_value=p_value*(i+1) / num_total_tests
        p_value_bh.append(bh_value)
    return (p_value_bh)

In [20]:
p_value_corr_bh=calc_benjamini_hochberg_corrections(a, num_total_tests)

In [21]:
p_sorted['p_value_corr_bh']=p_value_corr_bh

In [22]:
p_sorted.head()

,gene,sensitive_long_median,sensitive_short_median,mannwhiteney_pvalue,p_value_corr_bh
1043,hsa-mir-4761,0.000000,0.000000,0.000293,0.041252
149,hsa-mir-1301,126.137998,76.205448,0.000303,0.021341
29,hsa-mir-1180,225.496917,153.256471,0.000343,0.016102
1507,hsa-mir-6515,0.000000,0.000000,0.000665,0.023446
244,hsa-mir-1912,0.000000,0.000000,0.000815,0.022973


In [23]:
threshold = 0.05

significant_genes_bh = p_sorted[(p_sorted.p_value_corr_bh < threshold)].copy()
significant_genes_bh = significant_genes_bh.sort_values("p_value_corr_bh", ascending=True)
print(significant_genes_bh.shape[0])

176


In [391]:
id_name=[]
gene_name=[]

for x in ["-".join(x.strip().split("-")[:1])  for x in (significant_genes_bh['gene'])]:
    id_name.append(x)   
for x in ["-".join(x.strip().split("-")[1:])  for x in (significant_genes_bh['gene'])]:
    gene_name.append(x) 
    
significant_genes_bh['mirna_id']=id_name
significant_genes_bh['gene_symbol']=gene_name
significant_genes_bh=pd.DataFrame(significant_genes_bh)
significant_genes_bh.head()

,gene,resistant_median,sensitive_long_median,mannwhiteney_pvalue,mirna_id,gene_symbol,p_value_corr_bh
149,hsa-mir-1301,67.507454,126.137998,0.000022,hsa,mir-1301,0.002046
857,hsa-mir-4502,0.000000,0.000000,0.001244,hsa,mir-4502,0.016700
875,hsa-mir-4515,0.000000,0.641031,0.001110,hsa,mir-4515,0.017385
1754,hsa-mir-760,27.551583,56.722998,0.001605,hsa,mir-760,0.018857
780,hsa-mir-4435-1,0.000000,0.000000,0.001006,hsa,mir-4435-1,0.018921


In [392]:
significant_genes_bh.to_csv('new_data/miRNA/res_sl_bh.csv', sep=',', header=True, index=False)

## Holm 

In [142]:
#p_sorted=significant_genes.sort_values(by='mannwhiteney_pvalue')
p_sorted=pvalues.sort_values(by='mannwhiteney_pvalue')

In [143]:
a=p_sorted['mannwhiteney_pvalue'].get_values()
enumerate(a)

In [144]:
num_total_tests=len(a)
#num_total_tests=n_sens+n_senl

In [145]:
def calc_holm_corrections(p_values, num_total_tests):

   
    p_value_h = []
    for i, p_value in enumerate(p_values):
        h_value = p_value*(num_total_tests-i+1)
        p_value_h.append(h_value)
    return (p_value_h)

In [146]:
p_value_corr_h=calc_holm_corrections(a, num_total_tests)

In [147]:
p_sorted['p_value_corr_h']=p_value_corr_h
p_sorted.head()

,gene,resistant_median,sensitive_long_median,mannwhiteney_pvalue,p_value_corr_h
1533,hsa-mir-6719,0.0,0.0,0.491641,925.268526
409,hsa-mir-3156-1,0.0,0.0,0.491641,924.776885
410,hsa-mir-3156-2,0.0,0.0,0.491641,924.285244
417,hsa-mir-3160-2,0.0,0.0,0.491641,923.793603
420,hsa-mir-3163,0.0,0.0,0.491641,923.301961


In [148]:
threshold = 0.05

significant_genes_h = p_sorted[(p_sorted.p_value_corr_h < threshold)].copy()
significant_genes_h = significant_genes_h.sort_values("p_value_corr_h", ascending=True)
print(significant_genes_h.shape[0])

0


In [133]:
id_name=[]
gene_name=[]

for x in ["-".join(x.strip().split("-")[:1])  for x in (significant_genes_h['gene'])]:
    id_name.append(x)   
for x in ["-".join(x.strip().split("-")[1:])  for x in (significant_genes_h['gene'])]:
    gene_name.append(x) 
    
significant_genes_h['mirna_id']=id_name
significant_genes_h['gene_symbol']=gene_name
significant_genes_h=pd.DataFrame(significant_genes_h)
significant_genes_h.head()

,gene,resistant_median,sensitive_long_median,mannwhiteney_pvalue,p_value_corr_h,mirna_id,gene_symbol
149,hsa-mir-1301,67.507454,126.137998,0.000022,0.040963,hsa,mir-1301


## FDR other method

In [134]:
from statsmodels.stats.multitest import fdrcorrection

In [135]:
p_sorted=pvalues.sort_values(by='mannwhiteney_pvalue')

In [136]:
pvalues_FDR=pvalues['mannwhiteney_pvalue']
p_corrected=fdrcorrection(pvalues_FDR, alpha=0.05, method='indep')
w=p_corrected[1]

p_sorted['p_value_corr_FDR']=w
p_sorted.head()

,gene,resistant_median,sensitive_long_median,mannwhiteney_pvalue,p_value_corr_FDR
149,hsa-mir-1301,67.507454,126.137998,0.000022,0.550761
795,hsa-mir-4446,0.000000,0.082939,0.000821,0.550761
466,hsa-mir-3200,36.230009,79.711688,0.000857,0.550761
888,hsa-mir-4524a,0.000000,0.000000,0.000992,0.550761
780,hsa-mir-4435-1,0.000000,0.000000,0.001006,0.586241


In [137]:
threshold = 0.05

significant_genes_bh = p_sorted[(p_sorted.p_value_corr_FDR < threshold)].copy()
significant_genes_bh = significant_genes_bh.sort_values("p_value_corr_FDR", ascending=True)
print(significant_genes_bh.shape[0])

1
